In [1]:
!pip install xgboost lightgbm catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.2 MB/s eta 0:00:00


In [2]:
import numpy as np
from xgboost import XGBRanker
import lightgbm as lgb
from catboost import CatBoostRanker, Pool

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
np.random.seed(42)
X = np.random.rand(100, 5)
y = np.random.randint(1, 6, size=100)

group = [50, 50]

X_train, X_test = X[:80], X[80:]
y_train, y_test = y[:80], y[80:]
group_train = [40, 40]
group_test = [10, 10]

In [4]:
# Initialize and train the XGBoost Ranker
xgb_ranker = XGBRanker(
    objective="rank:pairwise",
    learning_rate=0.1,
    max_depth=3,
    n_estimators=100
)

xgb_ranker.fit(X_train, y_train, group=group_train)

# Predict with XGBoost Ranker
xgb_preds = xgb_ranker.predict(X_test)
print("XGBoost Ranking Predictions:", xgb_preds)

XGBoost Ranking Predictions: [-0.08559321  0.47115034  0.3079764   0.40871078  0.05832182 -0.2147222
 -0.07546412 -0.35630956  1.0136122  -1.4008616  -0.02610704  0.5294789
  0.17244184  0.61217964  0.06561965 -0.3144983   0.10000592 -0.05661608
  0.00811909  0.9806538 ]


In [5]:
# Prepare the dataset for LightGBM
lgb_train = lgb.Dataset(X_train, y_train, group=group_train)
lgb_test = lgb.Dataset(X_test, y_test, group=group_test, reference=lgb_train)

# LightGBM parameters
params = {
    "objective": "lambdarank",
    "metric": "ndcg",
    "learning_rate": 0.1,
    "max_depth": 3,
    "num_leaves": 31
}

# Train the LightGBM Ranker
lgb_ranker = lgb.train(
    params,
    lgb_train,
    valid_sets=[lgb_test]
)

# Predict with LightGBM Ranker
lgb_preds = lgb_ranker.predict(X_test)
print("LightGBM Ranking Predictions:", lgb_preds)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.053553 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 140
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 5
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wi

In [6]:
# Create Pool objects for CatBoost Ranker
train_pool = Pool(X_train, y_train, group_id=[0] * 40 + [1] * 40)
test_pool = Pool(X_test, y_test, group_id=[0] * 10 + [1] * 10)

# Initialize and train the CatBoost Ranker
catboost_ranker = CatBoostRanker(
    iterations=100,
    learning_rate=0.1,
    depth=3,
    verbose=10
)

catboost_ranker.fit(train_pool)

# Predict with CatBoost Ranker
catboost_preds = catboost_ranker.predict(test_pool)
print("CatBoost Ranking Predictions:", catboost_preds)

0:	total: 48.2ms	remaining: 4.78s
10:	total: 58.7ms	remaining: 475ms
20:	total: 70.9ms	remaining: 267ms
30:	total: 97.4ms	remaining: 217ms
40:	total: 123ms	remaining: 176ms
50:	total: 151ms	remaining: 145ms
60:	total: 179ms	remaining: 115ms
70:	total: 200ms	remaining: 81.6ms
80:	total: 213ms	remaining: 50ms
90:	total: 235ms	remaining: 23.2ms
99:	total: 260ms	remaining: 0us
CatBoost Ranking Predictions: [ 0.19971984 -0.18159392  0.62227024  1.12779662 -0.24247537 -0.85776596
 -1.19040211 -1.20013039  1.88685956 -1.73885655 -0.12818023  1.0459129
  0.22968846  0.84066966  1.21241688 -1.70950048  0.14430683 -0.38607107
  0.42418603  0.90027052]


In [7]:
# Store predictions in a dictionary
predictions = {
    'XGBoost': xgb_preds,
    'LightGBM': lgb_preds,
    'CatBoost': catboost_preds
}

for model, preds in predictions.items():
    print(f"{model} Predictions:", preds)

XGBoost Predictions: [-0.08559321  0.47115034  0.3079764   0.40871078  0.05832182 -0.2147222
 -0.07546412 -0.35630956  1.0136122  -1.4008616  -0.02610704  0.5294789
  0.17244184  0.61217964  0.06561965 -0.3144983   0.10000592 -0.05661608
  0.00811909  0.9806538 ]
LightGBM Predictions: [-0.84989444 -0.37332072 -0.13765569  1.16893411 -0.69280042 -1.46257798
 -0.44124369 -0.71177989  1.69539804 -2.05980036 -0.73827215  0.71156795
  0.61078154  0.28889737 -0.29477228 -0.6570429  -0.11016969 -0.56246439
 -0.09158867  1.20515679]
CatBoost Predictions: [ 0.19971984 -0.18159392  0.62227024  1.12779662 -0.24247537 -0.85776596
 -1.19040211 -1.20013039  1.88685956 -1.73885655 -0.12818023  1.0459129
  0.22968846  0.84066966  1.21241688 -1.70950048  0.14430683 -0.38607107
  0.42418603  0.90027052]
